<a href="https://colab.research.google.com/github/easyhardhoon/deep_learning_frameworks/blob/main/Pytorch3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#데이터 불러오기

2. 같은 클래스 별 폴더 이미지 데이터 이용



In [ ]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np
# ./class/tiger  ./class/lion 
# 만약 이런 폴더가 있다면. class 하위폴더에 tiger, lion 데이터 폴더가 있다면
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
#이렇게 한번에 모두를 처리할 수 있다는 개념이다. 
#class 하위의 이미지 데이터들을 tiger, lion 각각의 레이블에 맞춰 search 해준다. 
# tranf 으로 전처리까지 가능. 즉 모든 데이터를 불러오면서 각각의 레이블링까지 자동으로 매겨준다고 이해하자. 
trainloader = DataLoader(trainset, batch_size = 1, shuffle = False, num_workers=2)
print(len(trainLoader))

3. 개인 데이터 사용 (2 types) (직접 데이터 불러오기)

In [16]:
train_images = np.random.randint(256,size=(20,32,32,3))
train_labels = np.random.randint(2,size=(20,1))
#import preprocessing
#train_images, train_labels = preprocessing(train_images, train_labels)
#일종의 전처리 작업입니다. 앞의 transf 의 느낌과 비슷합니다.
print(train_images.shape, train_labels.shape)
#numpy에서는 (20,32,32,3) 이면 20의 batch size, 3개의 채널, 32 by 32 인풋 이미지 크기라고 생각하면 됩니다.

(20, 32, 32, 3) (20, 1)


In [17]:
class TensorData(Dataset): #이 문법은 Dataset 클래스를 상속받는 TensorData 클래스를 정의하는 것입니다.
  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.x_data = self.x_data.permute(0,3,1,2) #np.transpose 함수와 유사.  축 바꿔주기. 
    #pytorch에선 앞서 말했듯 (batch_size, channel, input_y,input_x) 이기 때문에 축을 바꿔주었다.
    self.y_data = torch.LongTensor(y_data) #그냥 Tensor도 있지만 이렇게 형식을 명확히 해주는 Tensor도 있습니다.
    self.len = self.y_data.shape[0] #x_data든 y_data든 데이터의 개수를 의미합니다. 
  def __getitem__(self,index):
      return self.x_data[index], self.y_data[index] #튜플형태의 데이터로 리턴
  def __len__(self):
    return self.len

#위의 클래스는

우리가 수동적으로 데이터를 가져올때 쓰는 클래스입니다.

정형화되어 있으니 형식을 외우면 좋을 것 같다.

In [18]:
train_data = TensorData(train_images, train_labels) #data를 정제한다! 수동으로. 
train_loader = DataLoader(train_data, batch_size=10, shuffle=True) #마지막으로 인공신경망에 넣기 위해 batch를 조정한다!
train_data[0][0].size()

torch.Size([3, 32, 32])

In [19]:
dataiter = iter(train_loader)
images,labels = dataiter.next()
images.size()

torch.Size([10, 3, 32, 32])

In [20]:
#전과정(파이토치 제공 데이터 사용할떄)
transf = tr.Compose([tr.Resize(8),tr.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf) #transform : 데이터 전처리
trainloader = DataLoader(trainset,batch_size=50,shuffle=True,num_workers=2) 

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


#정리

데이터를 불러오는 유형은 여러가지가 있다

파이토치 기본 제공 형태, 내장 폴더 형태, 개인 데이터 형태 이다.

어떤 유형에도 상관없이

**데이터 전처리 + 배치사이즈조정(최종관문)의 형태는 동일하다**

✅내장 폴더 형태의 데이터 불러오기를 잘 안하는 이유

    단 3줄이면 불러오기, 전처리, 배치사이즈 조정이 다 가능하지만 

    다른 작업과 공용으로 사용하거나 폴더가 아닌 SQL같은 곳에서 넘어오는 경우 등의

    폴더 정리를 못하는 경우에 이 작업을 수행하기 힘들다

    또한 파이토치에서 제공하는 transform의 종류가 제한적이다. 

    그래서 3번의 경우처럼 preprocessing 라이브러리를 import해서 1차 전처리를 해주고

    직접 tensor형변환 용 클래스를 만들어 2차 전처리를 해주고 데이터를 사용하는 편이 좋다

# TIP : 개인 데이터의 전처리를 transform 을 이용해서 할 수 있다.


In [21]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self):
    pass
  def __getitem__(self,index):
    pass
  def __len__(self):
    pass
  #이 양식이 기본 양식임을 기억/.

In [22]:
class MyDataset(Dataset): 
  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data 
    self.transform = transform
    self.len = len(y_data)
  def __getitem__(self,index):
      sample = self.x_data[index], self.y_data[index]
      if self.transform:
        sample= self.transform(sample) #여기서 transform이 형변환을 도와주게끔 설정. 실제로 하면 tensor로 자동변환
      return sample #그냥 리턴하면 numpy로 리턴
  def __len__(self):
    return self.len

#위 코드에서

앞의 TensorData 클래스와 비교해보면, 

transform을 인자로 받아서 transform이 켜지면 transform을 실행한후 리턴하는 모습을 알 수 있다.

TensorData(100프로 수동)에서는 tensor  변환을 클래스 내부에서 직접 구현했지만

MyDataSet(transform 이용)에서는 transform이 켜지면 transfrom이 알아서 실행하도록 도움을 받았다.

(transform이 tensor 형변환을 자동으로 해주었다)

In [23]:
#transform의 ToTensor,LInearTensor함수 직접 구현 (참고용입니다)
class ToTensor:
  def __call__(self,sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    return inputs, torch.LongTensor(labels)
class LinearTensor:
  def __init__(self,slope=1,bias=0):
    self.slope = slope
    self.bias = bias
  def __call__(self,sample):
    inputs,labels=sample
    inputs= self.slope*inputs+self.bias
    return inputs,labels

In [25]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)]) #참고로 여기는 tr.ToTensor 가 아닌 그냥 ToTensor입니다. 그냥 앞에서 만든걸 쓰겠다는 겁니다
ds1 = MyDataset(train_images,train_labels,transform=trans) #transform의 도움을 받을 것이니 trans을 앞에다가 선언해주는 모습
train_loader1 = DataLoader(ds1, batch_size=10,shuffle=True) #마지막 배치화는 모든 방법에서 동일하다

In [36]:
first_data  = ds1[0] #ds1의 첫번째 줄 입력이미지, 첫번쨰 줄 정답레이블을 묶어 의미합니다.
features, labels = first_data #기본적으로 numpy이든 tensor이든 tuple 형태기 때문에 이런 문법이 가능합니다.
print(type(features))
print(features.size()) #바로아래 코드와 동일한 뜻입니다.
ds1[0][0].size() #ds1의 첫번째 줄 입력이미지의 크기를 의미합니다. 배치를 하기 전이라 3,32,32가 나옵니다.
#참고로 totensor 구문에서 tensor로 바꾸고 차원도 바꿨기 때문에 numpy기준 차원이 pytorch에 맞게 올바르게 바뀌었씁니다.

<class 'torch.Tensor'>
torch.Size([3, 32, 32])


torch.Size([3, 32, 32])

**Tensor는 기본적으로 3차원 배열입니다**

In [35]:
dataiter = iter(train_loader1)
images,labels = dataiter.next()
images.size()

torch.Size([10, 3, 32, 32])

#다시 정리

✈직접 데이터를 불러오고 전처리할때(tr 도움없이)

data = __(train_images, train_labels) 

✈파이토치 기본 제공 데이터를 불러오고 정제할때

trans = tr.Compose([tr.Resize(8),tr.ToTensor()])

data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trans) 


✈직접 데이터를 불러오고 전처리할때(tr 도움받고)

trans = tr.Compose([ToTensor(), LinearTensor(2,5)])

data = __(train_images,train_labels,transform=trans) 

__는 사용자가 직접 설정한 데이터 처리 클래스입니다


⏰마지막 배치설정에는 모두가 똑같이

train_loader = DataLoader(data, batch_size = ??, shuffle = True)




#아무튼

이렇게 수동으로 ToTensor, LinearTensor.... 등의 전처리함수를 만들어서 tr.Compose([......]) 안에 넣어줄 수 있다. 

근데.... 굳이?? tr.ToTensor 쓰면 되는거 아닌가..??

처음으로 돌아가서

transf = tr.Compose([tr.Resize(8)]) 이게 실행이 되려면

여기에 들어오는 데이터가 PIL Image이어야 한다.

즉 PIL Image이어야 tr.ToTensor나 tr.Resize를 할 수 있다.

근데 여기서는 numpy이 이기 때문에 타입이 달라 실행이 되지 않는다

그래서 수동으로 ToTensor, LinearTensor를 만들어 준 것이다.

#그래도

일일이 수동으로 전처리함수를 만들기 귀찮다면???


In [46]:
class MyDataset(Dataset): 
  def __init__(self,x_data,y_data,transform=None):
    self.x_data = x_data
    self.y_data = y_data 
    self.transform = transform
    self.len = len(y_data)
  def __getitem__(self,index):
      sample = self.x_data[index], self.y_data[index]
      if self.transform:
        sample= self.transform(sample) #여기서 transform이 형변환을 도와주게끔 설정. 실제로 하면 tensor로 자동변환
      return sample #그냥 리턴하면 numpy로 리턴
  def __len__(self):
    return self.len

#여기서부터 주목
class MyTransform:
  def __call__(self,sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    labels = torch.FloatTensor(labels)
    #ToPILImage 함수를 통해 타입이 tensor에서 PIL 이미지로 바뀝니다.
    #tensor 에서 pil 이미지로 바꾸는 겁니다. numpy에서 바로 pil 이미지로 바꾸는 것은 아니다
    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor()]) 
    #다행히 topillimage 함수가 있다. 순서대로 수행되기 때문에 pil 이미지로 바뀌고 나서는 다음 resize수행 가능
    final_output = transf(inputs) #그냥 문법. 외우자. 이해하지말고
    return final_output, labels

In [47]:
#여기에 trans = tr.Compose([.....]) 가 없는 모습. 이유는 앞에서 설명함.  
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10,shuffle=True)
first_data = ds2[0]
features,labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [49]:
dataiter2 = iter(train_loader2)
images2,labels2 = dataiter2.next()
images2.size() #잘 바뀐 모습

torch.Size([10, 3, 128, 128])